In [1]:
from ase_interface import ANIENS
from ase_interface import ensemblemolecule_multigpu
from ase_interface import ensemblemolecule

import pyNeuroChem as pync
import pyaniasetools as pya
import hdnntools as hdt

import numpy as np
import  ase

import time

import os
os.environ["OMP_NUM_THREADS"] = "8"

import  ase
from ase.md.langevin import Langevin
from ase.md.verlet import VelocityVerlet
from ase.io.trajectory import Trajectory
from ase import units

from ase.optimize.fire import FIRE as QuasiNewton
from ase.optimize import LBFGS

from ase.md.nvtberendsen import NVTBerendsen
from ase.md import MDLogger

from ase.io import read, write

from ase.parallel import world

In [2]:
dir = '/home/jsmith48/scratch/MD_TEST/malonaldehyde/'

fname = 'malonaldehyde'

# Molecule file
molfile = dir + fname + '.xyz'

# Dynamics file
xyzfile = dir + 'mdcrd.xyz'

# Trajectory file
trajfile = dir + 'traj.dat'

# Optimized structure out:
optfile = dir + 'opt_' +  fname +'.xyz'

dt = 0.25
C = 0.0001 # Optimization convergence

ntdir = '/home/jsmith48/scratch/ccsd_extrapolation/final_train_models_3/ani-1x_dft_x8ens/'
cns = ntdir + 'rHCNO-5.2R_16-3.5A_a4-8.params'
sae = ntdir + 'sae_linfit.dat'
nnf = ntdir + 'train'
Nn = 8

In [3]:
#tvec = np.loadtxt('/home/jsmith48/scratch/carbon_nano_tube/new_test/'+fname+'.tvec',dtype=str)
#tvec = np.array(tvec[:,1:],dtype=np.float64)

In [4]:
#tvec

In [5]:
# Load molecule
mol = read(molfile)
# L = 40.0
# mol.set_cell(([[100.0, 0, 0],
#                [0, 35.0, 0],
#                [0, 0, 35.0]]))

#mol.set_cell((tvec))


#mol.set_pbc((True, True, True))
#mol.set_pbc((False, False, False))

# Set NC
aens = ensemblemolecule(cns, sae, nnf, Nn, 7)
#aens = ensemblemolecule(cns, sae, nnf, Nn, 1)

# Set ANI calculator
mol.set_calculator(ANIENS(aens))

s_time = time.time()
E = mol.get_potential_energy()
F = mol.get_forces()
print('Total time:',time.time()-s_time)

print(E)

spc = mol.get_chemical_symbols()
print(len(spc))

Total time: 0.029938697814941406
-7267.342930247066
9


In [6]:
# Optimize molecule
start_time = time.time()
dyn = LBFGS(mol)
dyn.run(fmax=C,steps=3000)
print('[ANI Total time:', time.time() - start_time, 'seconds]')

# Save optimized mol
spc = mol.get_chemical_symbols()
pos = mol.get_positions(wrap=False).reshape(1,len(spc),3)


hdt.writexyzfile(optfile, pos, spc)

       Step     Time          Energy         fmax
LBFGS:    0 10:10:58    -7267.342930        2.1936
LBFGS:    1 10:10:58    -7267.391873        1.4506
LBFGS:    2 10:10:58    -7267.409696        0.3359
LBFGS:    3 10:10:58    -7267.413622        0.1919
LBFGS:    4 10:10:58    -7267.416999        0.1713
LBFGS:    5 10:10:58    -7267.420144        0.2471
LBFGS:    6 10:10:58    -7267.421285        0.1401
LBFGS:    7 10:10:58    -7267.422400        0.1397
LBFGS:    8 10:10:58    -7267.423074        0.0933
LBFGS:    9 10:10:58    -7267.423331        0.0615
LBFGS:   10 10:10:58    -7267.423733        0.0795
LBFGS:   11 10:10:58    -7267.423996        0.0592
LBFGS:   12 10:10:58    -7267.424190        0.0593
LBFGS:   13 10:10:58    -7267.424315        0.0393
LBFGS:   14 10:10:58    -7267.424399        0.0340
LBFGS:   15 10:10:58    -7267.424434        0.0163
LBFGS:   16 10:10:58    -7267.424453        0.0144
LBFGS:   17 10:10:58    -7267.424463        0.0110
LBFGS:   18 10:10:58    -7267.42

In [7]:
# We want to run MD with constant energy using the VelocityVerlet algorithm.
#dyn = NVTBerendsen(mol, 1 * units.fs, 100.0, taut=1.0*1000*units.fs)
# coefficient to 0.04 atomic units.
dyn = Langevin(mol, dt * units.fs, 1.0 * units.kB, 0.04)

In [ ]:
# Open MD output
mdcrd = open(xyzfile,'w')

# Open MD output
traj = open(trajfile,'w')

# Define the printer
def storeenergy(a=mol, d=dyn, b=mdcrd, t=traj):  # store a reference to atoms in the definition.
    """Function to print the potential, kinetic and total energy."""
    epot = a.get_potential_energy() / len(a)
    ekin = a.get_kinetic_energy() / len(a)

    stddev =  hdt.evtokcal*a.calc.stddev
    Fstddev =  hdt.evtokcal*a.calc.Fstddev

    t.write(str(d.get_number_of_steps()) + ' ' + str(ekin / (1.5 * units.kB)) + ' ' + str(epot) + ' ' + str(ekin) + ' ' + str(epot+ekin) + ' ' + str(stddev) + '\n')
    b.write(str(len(a)) + '\n' + str(ekin / (1.5 * units.kB)) + ' Step: ' + str(d.get_number_of_steps()) + '\n')
    c = a.get_positions(wrap=False)
    for j, i in zip(a, c):
        b.write(str(j.symbol) + ' ' + str(i[0]) + ' ' + str(i[1]) + ' ' + str(i[2]) + '\n')

    print('Step: %d Energy per atom: Epot = %.3f  Ekin = %.3f (T=%.3fK)  '
          'Etot = %.6f Esig = %.3f Fsig = %.3f' % (d.get_number_of_steps(), epot, ekin, ekin / (1.5 * units.kB), epot + ekin, stddev,Fstddev))
    
# Set the printer
dyn.attach(storeenergy, interval=50)

In [ ]:
start_time = time.time()
dyn.set_temperature(300. * units.kB)
dyn.run(40000000)
print('[ANI Total time:', time.time() - start_time, 'seconds]')

Step: 50 Energy per atom: Epot = -807.490  Ekin = 0.002 (T=13.599K)  Etot = -807.488616 Esig = 0.337 Fsig = 0.117
Step: 100 Energy per atom: Epot = -807.488  Ekin = 0.003 (T=20.380K)  Etot = -807.485151 Esig = 0.373 Fsig = 0.116
Step: 150 Energy per atom: Epot = -807.487  Ekin = 0.006 (T=45.378K)  Etot = -807.481476 Esig = 0.327 Fsig = 0.116
Step: 200 Energy per atom: Epot = -807.485  Ekin = 0.008 (T=61.762K)  Etot = -807.476801 Esig = 0.354 Fsig = 0.124
Step: 250 Energy per atom: Epot = -807.480  Ekin = 0.006 (T=49.088K)  Etot = -807.473461 Esig = 0.430 Fsig = 0.119
Step: 300 Energy per atom: Epot = -807.485  Ekin = 0.010 (T=81.031K)  Etot = -807.474958 Esig = 0.363 Fsig = 0.137
Step: 350 Energy per atom: Epot = -807.482  Ekin = 0.009 (T=71.681K)  Etot = -807.473141 Esig = 0.352 Fsig = 0.115
Step: 400 Energy per atom: Epot = -807.482  Ekin = 0.011 (T=87.669K)  Etot = -807.470958 Esig = 0.438 Fsig = 0.120
Step: 450 Energy per atom: Epot = -807.482  Ekin = 0.011 (T=86.623K)  Etot = -807

Step: 3600 Energy per atom: Epot = -807.454  Ekin = 0.031 (T=241.926K)  Etot = -807.422775 Esig = 0.415 Fsig = 0.169
Step: 3650 Energy per atom: Epot = -807.450  Ekin = 0.026 (T=203.829K)  Etot = -807.424111 Esig = 0.303 Fsig = 0.169
Step: 3700 Energy per atom: Epot = -807.459  Ekin = 0.034 (T=266.278K)  Etot = -807.424279 Esig = 0.313 Fsig = 0.157
Step: 3750 Energy per atom: Epot = -807.450  Ekin = 0.031 (T=240.943K)  Etot = -807.419258 Esig = 0.394 Fsig = 0.129
Step: 3800 Energy per atom: Epot = -807.450  Ekin = 0.032 (T=243.842K)  Etot = -807.418214 Esig = 0.232 Fsig = 0.170
Step: 3850 Energy per atom: Epot = -807.469  Ekin = 0.050 (T=390.616K)  Etot = -807.418515 Esig = 0.364 Fsig = 0.138
Step: 3900 Energy per atom: Epot = -807.454  Ekin = 0.040 (T=308.133K)  Etot = -807.414188 Esig = 0.356 Fsig = 0.163
Step: 3950 Energy per atom: Epot = -807.444  Ekin = 0.023 (T=175.472K)  Etot = -807.421351 Esig = 0.343 Fsig = 0.142
Step: 4000 Energy per atom: Epot = -807.443  Ekin = 0.023 (T=181

Step: 7150 Energy per atom: Epot = -807.465  Ekin = 0.045 (T=344.413K)  Etot = -807.420475 Esig = 0.292 Fsig = 0.141
Step: 7200 Energy per atom: Epot = -807.443  Ekin = 0.019 (T=147.392K)  Etot = -807.424390 Esig = 0.352 Fsig = 0.156
Step: 7250 Energy per atom: Epot = -807.477  Ekin = 0.047 (T=365.954K)  Etot = -807.429600 Esig = 0.353 Fsig = 0.123
Step: 7300 Energy per atom: Epot = -807.463  Ekin = 0.035 (T=271.707K)  Etot = -807.428145 Esig = 0.421 Fsig = 0.159
Step: 7350 Energy per atom: Epot = -807.460  Ekin = 0.028 (T=217.627K)  Etot = -807.431808 Esig = 0.537 Fsig = 0.160
Step: 7400 Energy per atom: Epot = -807.465  Ekin = 0.033 (T=253.654K)  Etot = -807.431913 Esig = 0.466 Fsig = 0.146
Step: 7450 Energy per atom: Epot = -807.474  Ekin = 0.047 (T=361.120K)  Etot = -807.427270 Esig = 0.461 Fsig = 0.169
Step: 7500 Energy per atom: Epot = -807.451  Ekin = 0.025 (T=196.639K)  Etot = -807.425898 Esig = 0.294 Fsig = 0.139
Step: 7550 Energy per atom: Epot = -807.460  Ekin = 0.027 (T=212

Step: 10650 Energy per atom: Epot = -807.461  Ekin = 0.045 (T=347.944K)  Etot = -807.415912 Esig = 0.367 Fsig = 0.124
Step: 10700 Energy per atom: Epot = -807.451  Ekin = 0.036 (T=280.316K)  Etot = -807.414855 Esig = 0.346 Fsig = 0.173
Step: 10750 Energy per atom: Epot = -807.457  Ekin = 0.040 (T=309.326K)  Etot = -807.417488 Esig = 0.295 Fsig = 0.161
Step: 10800 Energy per atom: Epot = -807.469  Ekin = 0.048 (T=374.542K)  Etot = -807.420466 Esig = 0.307 Fsig = 0.133
Step: 10850 Energy per atom: Epot = -807.448  Ekin = 0.039 (T=300.728K)  Etot = -807.409560 Esig = 0.322 Fsig = 0.168
Step: 10900 Energy per atom: Epot = -807.458  Ekin = 0.045 (T=350.436K)  Etot = -807.412857 Esig = 0.398 Fsig = 0.151
Step: 10950 Energy per atom: Epot = -807.466  Ekin = 0.047 (T=366.364K)  Etot = -807.418526 Esig = 0.327 Fsig = 0.160
Step: 11000 Energy per atom: Epot = -807.456  Ekin = 0.044 (T=336.710K)  Etot = -807.412742 Esig = 0.361 Fsig = 0.133
Step: 11050 Energy per atom: Epot = -807.439  Ekin = 0.0

Step: 14150 Energy per atom: Epot = -807.454  Ekin = 0.045 (T=348.116K)  Etot = -807.408692 Esig = 0.413 Fsig = 0.157
Step: 14200 Energy per atom: Epot = -807.453  Ekin = 0.048 (T=372.635K)  Etot = -807.405143 Esig = 0.358 Fsig = 0.169
Step: 14250 Energy per atom: Epot = -807.445  Ekin = 0.044 (T=343.136K)  Etot = -807.400311 Esig = 0.251 Fsig = 0.126
Step: 14300 Energy per atom: Epot = -807.448  Ekin = 0.046 (T=352.029K)  Etot = -807.402296 Esig = 0.253 Fsig = 0.164
Step: 14350 Energy per atom: Epot = -807.431  Ekin = 0.027 (T=211.010K)  Etot = -807.403332 Esig = 0.285 Fsig = 0.146
Step: 14400 Energy per atom: Epot = -807.447  Ekin = 0.041 (T=316.371K)  Etot = -807.406429 Esig = 0.335 Fsig = 0.162
Step: 14450 Energy per atom: Epot = -807.429  Ekin = 0.035 (T=272.358K)  Etot = -807.394265 Esig = 0.432 Fsig = 0.147
Step: 14500 Energy per atom: Epot = -807.446  Ekin = 0.052 (T=399.714K)  Etot = -807.394773 Esig = 0.418 Fsig = 0.161
Step: 14550 Energy per atom: Epot = -807.443  Ekin = 0.0

...

Step: 1398850 Energy per atom: Epot = -807.474  Ekin = 0.029 (T=226.969K)  Etot = -807.444736 Esig = 0.342 Fsig = 0.131
Step: 1398900 Energy per atom: Epot = -807.477  Ekin = 0.038 (T=295.187K)  Etot = -807.438546 Esig = 0.451 Fsig = 0.127
Step: 1398950 Energy per atom: Epot = -807.468  Ekin = 0.034 (T=265.556K)  Etot = -807.433805 Esig = 0.295 Fsig = 0.146
Step: 1399000 Energy per atom: Epot = -807.459  Ekin = 0.030 (T=230.115K)  Etot = -807.428764 Esig = 0.469 Fsig = 0.130
Step: 1399050 Energy per atom: Epot = -807.452  Ekin = 0.025 (T=190.667K)  Etot = -807.427205 Esig = 0.319 Fsig = 0.141
Step: 1399100 Energy per atom: Epot = -807.454  Ekin = 0.026 (T=200.536K)  Etot = -807.428020 Esig = 0.371 Fsig = 0.125
Step: 1399150 Energy per atom: Epot = -807.452  Ekin = 0.019 (T=146.330K)  Etot = -807.433474 Esig = 0.320 Fsig = 0.140
Step: 1399200 Energy per atom: Epot = -807.465  Ekin = 0.030 (T=233.044K)  Etot = -807.435241 Esig = 0.346 Fsig = 0.133
Step: 1399250 Energy per atom: Epot = -8

Step: 1402300 Energy per atom: Epot = -807.458  Ekin = 0.050 (T=383.300K)  Etot = -807.408193 Esig = 0.360 Fsig = 0.138
Step: 1402350 Energy per atom: Epot = -807.455  Ekin = 0.045 (T=345.135K)  Etot = -807.410503 Esig = 0.301 Fsig = 0.150
Step: 1402400 Energy per atom: Epot = -807.435  Ekin = 0.022 (T=169.590K)  Etot = -807.413093 Esig = 0.428 Fsig = 0.140
Step: 1402450 Energy per atom: Epot = -807.453  Ekin = 0.040 (T=306.256K)  Etot = -807.413151 Esig = 0.330 Fsig = 0.149
Step: 1402500 Energy per atom: Epot = -807.457  Ekin = 0.046 (T=354.245K)  Etot = -807.411029 Esig = 0.300 Fsig = 0.141
Step: 1402550 Energy per atom: Epot = -807.440  Ekin = 0.030 (T=235.804K)  Etot = -807.409380 Esig = 0.365 Fsig = 0.128
Step: 1402600 Energy per atom: Epot = -807.454  Ekin = 0.048 (T=371.300K)  Etot = -807.406374 Esig = 0.389 Fsig = 0.139
Step: 1402650 Energy per atom: Epot = -807.465  Ekin = 0.062 (T=480.868K)  Etot = -807.402552 Esig = 0.419 Fsig = 0.164
Step: 1402700 Energy per atom: Epot = -8

Step: 1405750 Energy per atom: Epot = -807.458  Ekin = 0.026 (T=201.624K)  Etot = -807.432187 Esig = 0.383 Fsig = 0.152
Step: 1405800 Energy per atom: Epot = -807.455  Ekin = 0.028 (T=213.353K)  Etot = -807.427626 Esig = 0.352 Fsig = 0.128
Step: 1405850 Energy per atom: Epot = -807.466  Ekin = 0.034 (T=264.550K)  Etot = -807.431351 Esig = 0.339 Fsig = 0.170
Step: 1405900 Energy per atom: Epot = -807.468  Ekin = 0.037 (T=289.065K)  Etot = -807.430853 Esig = 0.387 Fsig = 0.134
Step: 1405950 Energy per atom: Epot = -807.470  Ekin = 0.036 (T=280.542K)  Etot = -807.433641 Esig = 0.490 Fsig = 0.126
Step: 1406000 Energy per atom: Epot = -807.473  Ekin = 0.042 (T=327.867K)  Etot = -807.430979 Esig = 0.287 Fsig = 0.158
Step: 1406050 Energy per atom: Epot = -807.470  Ekin = 0.037 (T=285.845K)  Etot = -807.433517 Esig = 0.312 Fsig = 0.118
Step: 1406100 Energy per atom: Epot = -807.464  Ekin = 0.033 (T=254.283K)  Etot = -807.431100 Esig = 0.310 Fsig = 0.152
Step: 1406150 Energy per atom: Epot = -8

Step: 1409200 Energy per atom: Epot = -807.459  Ekin = 0.053 (T=411.375K)  Etot = -807.405510 Esig = 0.319 Fsig = 0.138
Step: 1409250 Energy per atom: Epot = -807.456  Ekin = 0.043 (T=333.473K)  Etot = -807.413270 Esig = 0.299 Fsig = 0.138
Step: 1409300 Energy per atom: Epot = -807.445  Ekin = 0.031 (T=236.944K)  Etot = -807.414835 Esig = 0.372 Fsig = 0.133
Step: 1409350 Energy per atom: Epot = -807.448  Ekin = 0.037 (T=283.986K)  Etot = -807.411629 Esig = 0.431 Fsig = 0.158
Step: 1409400 Energy per atom: Epot = -807.452  Ekin = 0.042 (T=321.602K)  Etot = -807.410840 Esig = 0.509 Fsig = 0.116
Step: 1409450 Energy per atom: Epot = -807.455  Ekin = 0.046 (T=355.281K)  Etot = -807.409352 Esig = 0.310 Fsig = 0.148
Step: 1409500 Energy per atom: Epot = -807.465  Ekin = 0.056 (T=432.131K)  Etot = -807.409356 Esig = 0.233 Fsig = 0.109
Step: 1409550 Energy per atom: Epot = -807.464  Ekin = 0.052 (T=403.466K)  Etot = -807.412166 Esig = 0.263 Fsig = 0.129
Step: 1409600 Energy per atom: Epot = -8

Step: 1412650 Energy per atom: Epot = -807.462  Ekin = 0.023 (T=177.601K)  Etot = -807.439541 Esig = 0.293 Fsig = 0.143
Step: 1412700 Energy per atom: Epot = -807.474  Ekin = 0.035 (T=272.566K)  Etot = -807.438733 Esig = 0.369 Fsig = 0.128
Step: 1412750 Energy per atom: Epot = -807.459  Ekin = 0.022 (T=172.888K)  Etot = -807.436516 Esig = 0.332 Fsig = 0.158
Step: 1412800 Energy per atom: Epot = -807.467  Ekin = 0.031 (T=242.102K)  Etot = -807.435773 Esig = 0.252 Fsig = 0.141
Step: 1412850 Energy per atom: Epot = -807.466  Ekin = 0.026 (T=203.212K)  Etot = -807.439764 Esig = 0.289 Fsig = 0.129
Step: 1412900 Energy per atom: Epot = -807.472  Ekin = 0.033 (T=252.960K)  Etot = -807.439124 Esig = 0.421 Fsig = 0.146
Step: 1412950 Energy per atom: Epot = -807.466  Ekin = 0.026 (T=200.744K)  Etot = -807.440123 Esig = 0.298 Fsig = 0.132
Step: 1413000 Energy per atom: Epot = -807.463  Ekin = 0.019 (T=144.917K)  Etot = -807.444437 Esig = 0.499 Fsig = 0.160
Step: 1413050 Energy per atom: Epot = -8